# **Model Server**

_______________________________________________________________________________

<a id="top"></a>
## **steps**
**[notebook installs](#installs)**<br>
**[nuclio code section](#nuclio)**<br>
    - [inference server](#server)<br>
**[deploy](#deploy)**<br>
**[test deployment](#test)**<br>
**[test saved model object](#infderence)**


<a id="installs"></a>
_______________________________________________________________________________

## **notebook installs**

These are the packages we'll need to run this notebook, please install them once:

    !pip install -U kfserving==0.2.0 tensorflow==2.0.0b1 keras pandas 
    !pip install -U kubernetes==9.0.0
    !pip install -U git+https://github.com/mlrun/mlrun.git@development
    !pip install -U azure

In [1]:
#     !pip install -U git+https://github.com/yjb-ds/functions-demo.git
#     !pip install -U git+https://github.com/mlrun/mlrun.git@development

<a id="nuclio"></a>
_______________________________________________________________________________

## **nuclio code section**

If you have never run nuclio functions in your notebooks, please uncomment and run the following:

    !pip install nuclio-jupyter

In [2]:
# nuclio: ignore
import nuclio

Install the following packages available to the function:

In [3]:
%%nuclio cmd -c
pip install -U kfserving==0.2.0 azure numpy tensorflow==2.0.0b1 keras joblib
pip install -U git+https://github.com/mlrun/mlrun.git@development
pip install -U git+https://github.com/yjb-ds/functions-demo.git

<a id="server"></a>
### _**inference server**_

In [4]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [5]:
import kfserving
import os
import numpy as np
import joblib
from typing import List
from tensorflow.keras.models import Sequential

from functions.models import FeaturesEngineer, classifier_gen
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

In [6]:
TARGET_PATH = '/User/mlrun/simdata'

In [7]:
class TFKerasClassifier(kfserving.KFModel):
    def __init__(self,
                 name: str,
                 model_dir: str,
                 classifier: Sequential = None):
        """TFKerasClassifier
        
        KubeFlow serving model wrapper.
        
        :param name:            model name
        :param model_dir: path of stored model
        :param classifier:      class type of classifier model
        
        """
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if classifier:
            self.ready = True

    def load(self):
        """Load model from KubeFlow storage.
        """
        ffg = FeaturesGenerator()

        # scaler
        ss = StandardScaler()
        ss.__dict__ = joblib.load(f'{model_dir}/scaler.pickle')

        # keras model
        ksm = classifier_gen()
        ksm.load_weights(f'{model_dir}/weights.h5')

        pipe = make_pipeline(ffg, ss, ksm)

#       pipe = os.path.join(
#             kfserving.Storage.download(self.model_dir), 'scaler.pickle')

        self.classifier = pipe

    def predict(self, body: List) -> List:
        """Generate model predictions from sample.
        
        :param body: A list of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body)
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [8]:
# nuclio: end-code

<a id="deploy"></a>
_______________________________________________________________________________

## **deploy**

In [9]:
from mlrun import new_model_server

In [11]:
fn = new_model_server("tfkeras-serving", 
                      models={"tfkeras_joblib": TARGET_PATH}, 
                      model_class="TFKerasClassifier")
fn.with_v3io("User","~/")

In [12]:
#fn.verbose=True
addr = fn.deploy(project="refactor-demos")

[mlrun] 2019-12-30 11:15:11,489 deploy started
[nuclio] 2019-12-30 11:15:12,566 (info) Building processor image
[nuclio] 2019-12-30 11:18:28,454 (info) Build complete
[nuclio] 2019-12-30 11:18:38,628 (info) Function deploy complete
[nuclio] 2019-12-30 11:18:38,633 done creating tfkeras-serving, function address: 3.136.3.102:30972


<a d="test"></a>
_______________________________________________________________________________

## **test deployment endpoint**

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

import requests

In [ ]:
# Grab some data from the test set
xtest = pq.read_table("/User/projects/paysim/data/xtest.parquet").to_pandas()
ytest = pq.read_table("/User/projects/paysim/data/ytest.parquet").to_pandas()

In [ ]:
# Seldon protocol event
event_seldon = {
    "data" : {
        "ndarray": features.values.tolist()
    }
}
csel = str(event_seldon).replace("\"", "\"").replace("\n", "").replace(" ", "")

In [ ]:
resp = requests.put(addr + "/predict/tfkeras_joblib", data=csel)
print(resp.content)

<a id="inference-test"></a>
_______________________________________________________________________________

## **test saved model object**

Here we run a test outside of mlrun and the context: grab the estimated model file created on the **[kubeflow pipeline](kubeflow-pipeline.ipynb)**, load, and run a test matrix through it.

In [ ]:
import joblib
model_file = f"/User/projects/paysim/{CLASSIFIER_FILE}"
tfkeras_model = joblib.load(open(model_file, "rb"))

#### _**test one row**_

In [ ]:
testvals = [1.86888885,1.57777536,-0.39419246,2.05800271,-0.75799805,1.57381201,1.4090718,1.26302791,1.08653808,1.22390795,0.46826237,0.78775787,0.,0.84368491,0.33878541,-0.4717221,2.54822445,0.38823441,-0.57677436,1.70674813,0.,0.8656919,0.87541604,0.98195601,2.04881859,1.63834357,1.42886198,1.33937621]
testvals = np.asarray(testvals).reshape(-1,1).T

In [ ]:
lgbm_model.predict(testvals)

#### _**test matrix**_

In [ ]:
lgbm_model.predict(features)

In [ ]:
!mlrun clean